In [1]:
from ipymlwidgets.widgets import Image
from ipymlwidgets.utils import demo_image


image = Image(demo_image())
display(image)

print(image.image.shape)

image[10:20, 10:20] = [255, 0, 0]



Image(height=32, width=32)

(32, 32, 3)
